# LSTM Time Series Deep Learning and Adversarial Attacks

## 1. Project Background Information

This project reproduces and expands upon work published in [1] and [2] on Long Short-Term Memory (LSTM) predictive models and adversarial attacks on those models.  The previous studies used Long Short-Term Memory (LSTM) time series classification models trained with data from the Medical Information Mart for Intensive Care (MIMIC-III) database to predict Intensive Care Unit (ICU) patient outcomes. Input features to the classification models consisted of 13 lab measurements and 6 vital signs. A binary variable representing in-hospital mortality was the prediction target.

In [1], an adversarial attack algorithm was used to identify small perturbations which, when applied to a real, correctly-classified input features, caused a trained model to misclassify the perturbed input. L1 regularization was applied to the adversarial attack loss function to favor adversarial examples with sparse perturbations that resemble the structure of data entry errors most likely to occur in real medical data. Samples were attacked serially (one a time), and the attack process on a sample was stopped upon finding a single adversarial perturbation to that samples input features. After attacking a full dataset, susceptibility calculations were  performed to identify input feature space regions most vulnerable to adversarial attack.

The current study follows an approach similar to that of the previous studies. We use the same dataset, input features, and prediction targets to train a LSTM binary classification model and subsequently search for adversarial examples using an L1 regularized attack algorithm. Aspects of the current work that expand upon the previous studies include a vectorized (faster) approach to data preprocessing, extensive hyperparameter tuning (of both the predictive model and attack algorithm), improved performance of the predictive model, implementation of a GPU-compatible attack algorithm that enables attacking samples in batches, and not halting the attack process upon finding a single adversarial perturbation for a sample (so that additional, lower loss adversarial perturbations can be discovered).

## 2. Development Environment Setup

### 2.1 Docker Container

The code and instructions in this notebook assume the development environment has been set up by completing all steps in the [How to run this project](https://github.com/duanegoodner/lstm_adversarial_attack/tree/main#3-how-to-run-this-project) section of the roject [README](https://github.com/duanegoodner/lstm_adversarial_attack). If you have used the procedure described there to run this notebook inside a `lstm_aa_app` Docker container, then the output of the following code cell should be `PosixPath('/home/devspace/project/notebooks')`.

In [1]:
from pathlib import Path
Path.cwd()

PosixPath('/home/devspace/project/notebooks')

We can also check the contents of the container project root directory

In [2]:
!ls /home/devspace/project

README.md  config  data  docker  docs  notebooks  src


If `/home/devspace/project` is correctly mapped to your local project root, the above output should match the list of files in the local project root.

## 3. Project File Structure
All .py files are in `/home/devspace/project/src/lstm_adversarial_attack`. This directory contains four sub-packages responsible for different parts of the project data pipeline (`query_db`, `preprocess`, `tune_train`, `attack`, and `attack_analysis`). The code snippets in this notebook instantiate classes and call methods of files under the `src` directory. Look to the code and docstrings there for implementation details. 


Data files are under `/home/devspace/project/data/` in subdirectories with names that match the sub-package names.



## 4. Imports


### 4.1 Standard Library and External Packages
Most of the necessary standard library imports and external package imports are handled code in the `src` sub-packages, but we need to import a few things here.

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pprint
import pandas as pd
import sys
import torch
from IPython.display import Markdown as md
from torch.utils.data import Dataset, random_split

### 4.2 Internal Project Sub-packages and Modules
To make it easier to understand what various internal packages and modules do, we will wait to import them until just before the notebook code cells where they are first used. For now, we import the `src` path defined in `lstm_adversarial_attack/notebooks/src_path`, add it to sys.path (so we can easily import project code). We also import project config files.

In [4]:
import src_paths
sys.path.append(str(src_paths.lstm_adversarial_attack_pkg))
import lstm_adversarial_attack.config_paths as cfg_paths
import lstm_adversarial_attack.config_settings as cfg_set

### 5. Check for GPU

We won't need a GPU until we reach the HyperParameter Tuning section, but it is good to find out now we have a GPU that PyTorch can use. If we do not have one, we likely do not want to try to run the project.

In [5]:
import torch

if torch.cuda.is_available():
    cur_device = torch.device("cuda:0")
else:
    cur_device = torch.device("cpu")

print(f"cur_device is {cur_device}")

cur_device is cuda:0


## 5. Database Queries

### 5.1 `.sql` files
To obtain the necessary raw data, we will use modified versions of files (originally intended for Google Big Query) from https://github.com/MIT-LCP/mimic-code/tree/main/mimic-iii/concepts/pivot. The paths to the `.sql` query files are stored in a list as `DB_QUERIES` in the project `config_paths` file:

In [6]:
pprint.pprint(cfg_paths.DB_QUERIES)

[PosixPath('/home/devspace/project/src/mimiciii_queries/icustay_detail.sql'),
 PosixPath('/home/devspace/project/src/mimiciii_queries/pivoted_bg.sql'),
 PosixPath('/home/devspace/project/src/mimiciii_queries/pivoted_lab.sql'),
 PosixPath('/home/devspace/project/src/mimiciii_queries/pivoted_vital.sql')]


### 5.2 Connecting to Database and Executing Queries

To connect to the database, and execute the queries, we instantiate a `MimiciiiDatabaseAccess` object and use its .connect(), .run_sql_queries() and .close_connection() methods.

In [ ]:
import lstm_adversarial_attack.query_db.mimiciii_database as mdb

db_access = mdb.MimiciiiDatabaseAccess(
    dotenv_path=cfg_paths.DB_DOTENV_PATH, output_dir=cfg_paths.DB_OUTPUT_DIR
)
db_access.connect()
db_query_results = db_access.run_sql_queries(
    sql_query_paths=cfg_paths.DB_QUERIES
)
db_access.close_connection()

The results of each `.sql` query is saved to a `.csv` file. The path to each of these files is shown in the terminal output above. The output path of the queries is defined by variable `DB_OUTPUT_DIR` in `config_settings`.

## 6. Preprocess

The `preprocess` sub-package converts `.sql` query output into arrays that can then be easily converted into the tensors used by a PyTorch model. 


### 6.1 Instantiate a Preprocessor object
We import the `preprocessor` module, instantiate a `Preprocessor` object, and examine its .preprocess_modules data member. 

In [8]:
import lstm_adversarial_attack.preprocess.preprocessor as pre
preprocessor = pre.Preprocessor()

Wer can print the `Preprocessor` object's `.preprocessor_modules`attribute contents:

In [9]:
pprint.pprint([item.__class__ for item in preprocessor.preprocess_modules])

[<class 'lstm_adversarial_attack.preprocess.prefilter.Prefilter'>,
 <class 'lstm_adversarial_attack.preprocess.icustay_measurement_combiner.ICUStayMeasurementCombiner'>,
 <class 'lstm_adversarial_attack.preprocess.sample_list_builder.FullAdmissionListBuilder'>,
 <class 'lstm_adversarial_attack.preprocess.feature_builder.FeatureBuilder'>,
 <class 'lstm_adversarial_attack.preprocess.feature_finalizer.FeatureFinalizer'>]


* Prefilter reads the database query outputs into Pandas Dataframes, removes all data related to patients younger than 18 years in age, ensures consistent column naming formats, and takes care of datatype details.
* ICUStayMeasurementCombiner performs various joins (aka "merges" in the language of Pandas) to combine lab and vital sign measurement data with ICU stay data.
* FullAdmissionListBuilder generates a list consisting of one FullAdmissionData object per ICU stay. The attributes of a FullAdmissionData object include ICU stay info, and a dataframe containing the measurement and timestamp data for all vital sign and lab data associated with the ICU stay.
* FeatureBuilder resamples the time series datafame to one-hour intervals, imputes missing data, winsorizes measurement values (with cutoffs at the 5th and 95th global percentiles), and normalizes the measuremnt values so all data are between 0 and 1.
* FeatureFinalizer selects the data observation time window (default starts at hospital admission time and ends 48 hours after admission). This module outputs the entire dataset features as a list of numpy arrays, and the mortality labels as a list of integers. These data structures (saved as .pickle files) will be convenient starting points when the `tune_train` and `attack` sub-packages need to create PyTorch Datasets.

Now that we have a some background info, we are ready to run the Preprocessor.

In [ ]:
preprocessed_resources = preprocessor.preprocess()

## 7. Dataset

### 7.1 Create the Pytorch Dataset object
We import module `x19_mort_general_dataset` and use it along with files saved by the `preprocessor.feature_finalizer` to insantiate a Pytorch Dataset

In [10]:
import lstm_adversarial_attack.x19_mort_general_dataset as xmd
dataset = xmd.X19MGeneralDataset.from_feature_finalizer_output()

### 7.2 Examine the dataset

Next we instantiate a DatasetInspector from the `x19_mort_general_dataset` module and use its methods to display some basic information about the dataset.

In [11]:
dataset_inspector = xmd.DatasetInspector(dataset=dataset)
dataset_inspector.view_basic_info()
dataset_inspector.view_seq_length_summary()
dataset_inspector.view_label_summary()

There are 41951 samples in the Dataset.
Calling `__getitem__` on the Dataset returns a tuple of length 2.
The first element of this tuple is a 2-D Tensor with 19 columns and data type torch.float32.
The second element is a 0-D Tensor with data type torch.int64


seq_length,6,13,14,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48
num_samples,1,1,1,2,4,3,8,12,25,49,84,144,126,110,93,95,84,90,75,99,98,113,148,152,189,199,220,178,231,203,211,191,185,221,474,37832


class_label,0,1
nunm_samples,37338,4613


Each item in the dataset is from a unique ICU stay. The input features for a single sample are represented by a 2D tensor where the column corresponds to a particular lab or vital sign measurement, and the row corresponds to time in hours after hospital admission. All samples' input feature tensors have the same number of columns, but the number of rows can vary from sample-to-sample. In LSTM lingo, the number of time steps assiciated with a sample is called the *sequence length*. In the current analysis, the Preprocessor removed all measuremens > 48 hours post-admission, so the maximum sequence length is 48. Samples ICU stays with < 48 hours of observations have smaller sequence lengths.

A class label of 1 corresponds to an in-hospital mortality event. Less than 15% of samples belong this class. We will need to take the class imabalance into account when tuning and training predictive models with this dataset. 

## 8. Model Architecture

The starting point for our predictive model is based on the model in [1] and consists of the following layers:

| Layer # | Description        | Input Shape                            | Parameters          | Output Shape           | Activation       |
| ------- | ------------------ | -------------------------------------- | ------------------- | ---------------------- | ---------------- |
| 1       | Bidirectional LSTM | (b, t<sub>max</sub> = 48, n<sub>meas</sub> = 19) | n<sub>LSTM</sub>    | (b, 2n<sub>LSTM</sub>) | a<sub>LSTM</sub> |
| 2       | Dropoout           | (b, 2n<sub>LSTM</sub>)                 | P<sub>dropout</sub> | (b, 2n<sub>LSTM</sub>) | -                |
| 3       | Fully Connected    | (b, 2n<sub>LSTM</sub>)                 | n<sub>FC</sub>      | (b, n<sub>FC</sub>)    | a<sub>FC</sub>   |
| 4       | Output             | (b, n<sub>FC</sub>)                    | n<sub>out</sub> = 2 | (b, n<sub>out</sub>    | a<sub>out</sub>  |


The parameters from the above table are defined as:

| Parameter           | Description                                             |
| ------------------- | ------------------------------------------------------- |
| b                   | Batch size                                              |
| t<sub>max</sub>     | Maximum input sequence length                           |
| n<sub>meas</sub>    | Number of patient measurement types                     |
| n<sub>LSTM</sub>    | Number of features in a LSTM hidden state               |
| a<sub>LSTM</sub>    | Activation function for the LSTM output                 |
| P<sub>dropout</sub> | Dropout probablity                                      |
| n<sub>FC</sub>      | Numbef of nodes in the fully connected layer            |
| a<sub>FC</sub>      | Activation function for the fully connected layer ouput |
| n<sub>out</sub>     | Number of nodes in the output layer                     |
| a<sub>out</sub>     | Activation function for the output layer                |


Note that n<sub>meas</sub>, n<sub>out</sub>, abd s<sub>max</sub> are fixed. We have chosen to always use all 19 patient measurement types, and our classification problem always has two classes. In our current data pipeline, data collected outside of a specified time window are removed during the final preprocessing phase. If we want the observation window to be tunable, it would be helpful to move the `preprocess.feature_finalizer` module into the `tune_attack` sub-package.

## 9. Hyperparameter Tuning

### 9.1 Architectural hyperparameters

The following table lists the ranges architectural parameters to be explored during hyperparameter tuning.

| Parameter           | Tuning Type  | Values                            |
| ------------------- | ------------ | --------------------------------- |
| b                   | Discrete     | 2<sup>k</sup> , k = 5, 6, 7, 8    |                    
| h<sub>LSTM</sub>    | Discrete     | 2<sup>k</sup> , k = 5, 6, 7       |
| a<sub>LSTM</sub>    | Discrete     | ReLU, Tanh                        |
| P<sub>dropout</sub> | Continuous   | 0.000 $\textemdash$ 0.5000        |
| h<sub>FC</sub>      | Discrete     | 2<sup>k</sup> , k = 4, 5, 6, 7, 8 |
| a<sub>FC</sub>      | Discrete     | ReLU, Tanh                        |


### 9.2 Trainer hyperparameters




During hyperparameter tuning, we also explore different training optimization algorithms and learning rates.

| Parameter     | Tuning Type | Values             |
| ------------- | ----------- | ------------------ |
| Optimizer     | Discrete    | SGD, RMSprop, Adam |
| Learning Rate | Continuous  | 1e-5 - 1e-1        |

When using the Adam optimizer, we always use the Pytorch default values of $\beta_1 = 0.9, \beta_2 = 0.999, \epsilon = 10^{-8}$. 

### 9.3 Implementation Details
The `HyperParameterTuner` class in the `lstm_adversarial_attack.tune_train` sub-package implements a Stratified K-fold (default K = 5) cross-validation tuning scheme that utilizes the Optuna framework. When selecting samples for each fold, we oversample from samples in the minority class (label = 1). For a given set of hyperparameters, the `HyperParameterTUner.objective_fn()` returns the mean validation loss across the K folds, and this mean loss is used as a minimization target by an Optuna `TPESampler` to select new sets of hyperparameters for additional trials. `HyperParameterTuner` also uses an Optuna `MedianPruner` to stop unpromising trials early.



### 9.4 Instantiate and Examine TunerDriver

In [13]:
import lstm_adversarial_attack.tune_train.tuner_driver as td
tuner_driver = td.TunerDriver(
    device=cur_device,
    continue_study_path=cfg_paths.ONGOING_TUNING_STUDY_PICKLE,
    output_dir=cfg_paths.ONGOING_TUNING_STUDY_DIR,)

The `continue_study_path` and `output_dir` arguments passed to the `TunerDriver` constructor will allow us to build upon an existing Optuna study that contains learning from previously run trials. Model hyperparameter tuning is the most time-consuming part of the overall project pipeline, so we usually do not want to start from scratch. (But if you really want to start from scratch, just don't pass either of these path arguments to the `TuneDriver` constructor). For reference, the values of the two path variables we passed to the TunerDriver constructor are:

In [14]:
print(f"continue_study_path = {cfg_paths.ONGOING_TUNING_STUDY_PICKLE}")
print(f"output_dir = {cfg_paths.ONGOING_TUNING_STUDY_DIR}")

continue_study_path = /home/devspace/project/data/tune_train/hyperparameter_tuning/continued_trials/checkpoints_tuner/optuna_study.pickle
output_dir = /home/devspace/project/data/tune_train/hyperparameter_tuning/continued_trials


The TunerDriver object has many default parameters/attributes set by values in `cfg_paths` and `cfg_settings`. Note that its `.tuner` attribute is a `HyperParameterTuner` which in turn has a `.tuning_ranges` attribute that specifies our hyperparameter search space.

In [15]:
print(f"The tuner_driver.tuner is a {type(tuner_driver.tuner)}\n")
print("tuner_driver.tuner.tuning_ranges:")
pprint.pprint(tuner_driver.tuner.tuning_ranges)

The tuner_driver.tuner is a <class 'lstm_adversarial_attack.tune_train.hyperparameter_tuner.HyperParameterTuner'>

tuner_driver.tuner.tuning_ranges:
X19MLSTMTuningRanges(log_lstm_hidden_size=(5, 7),
                     lstm_act_options=('ReLU', 'Tanh'),
                     dropout=(0, 0.5),
                     log_fc_hidden_size=(4, 8),
                     fc_act_options=('ReLU', 'Tanh'),
                     optimizer_options=('Adam', 'RMSprop', 'SGD'),
                     learning_rate=(1e-05, 0.1),
                     log_batch_size=(5, 8))


### 9.5 Run the TunerDriver
The code in the next cell will run the Tuner Driver (and its associated HyperParameterTuner). Before starting, a few things to note:
* Depending your GPU compute power, running the full 30 trials could take 2 - 20 hours.
* If the study is stopped early (via CTRL-C or the Jupyter Stop button), learning from whatever trials have completed up to that point will be saved.
* While the tuning trials are running, look ahead to the next Markdown cell for instructions on how to monitor progress in Tensorboard (depending on your notebook output settings you may need to scroll down to see that cell)

In [ ]:
my_completed_study = tuner_driver.run(num_trials=30)

### 9.6 Monitor Tuning Progress with Tensorboard

While we are tuning hyperparameters, we can monitor results in Tensorboard. We can run tensorboard by starting a zsh session inside the project app container, and launching the tensorboard server from there:
```
$ docker exec -it lstm_aa_app_dev /bin/zsh
> tensorboard --logdir=/home/devspace/project/data/hyperparameter_tuning/continued_trials/tensorboard --host=0.0.0.0
```
Then, in your browser, go to: `http://localhost:6006/`

You should see something like the screenshot below.  The x-axis for all plots is epoch number. (Unfortunately, there is no good way to add axis labels in Tensorboard.)

In this example we are in the middle of running trial #21. Trial #20 completed the default number of epochs per fold (100). Trial #19 only ran 20 epochs because it was pruned by the Optuna `MeadianPruner`. 

![tensorboard_image](images/tensorboard_hyperparameter_tuning.png)



### 9.7 Select Final Hyperparameters
When we are done tuning, we can view our best set of hyperparameters by examining the `Optuna.Study` object from our above tuning run(s).


In [17]:
import lstm_adversarial_attack.resource_io as rio
study = rio.ResourceImporter().import_pickle_to_object(
    path=cfg_paths.ONGOING_TUNING_STUDY_PICKLE
)

print(f"The best trial result is from trial # {study.best_trial.number}.\n")
print("The set of hyperparameters from this trial are:")
pprint.pprint(study.best_params)


The best trial result is from trial # 20.

The set of hyperparameters from this trial are:
{'dropout': 0.029018875280141854,
 'fc_act_name': 'Tanh',
 'learning_rate': 0.0002784280532512521,
 'log_batch_size': 5,
 'log_fc_hidden_size': 4,
 'log_lstm_hidden_size': 7,
 'lstm_act_name': 'Tanh',
 'optimizer_name': 'Adam'}


### 9.8 Run K-Fold Cross Validation with "Best" Hyperparameters and Extended Training (More Epochs)
In the above tuning runs, we only run 100 epochs per fold (in the interest of reducing compute requirements). Based on the validation loss and AUC curves, it appears that we could improve our predictive performance (i.e. decrease validation loss, and increase AUC) by training longer. We now run another round of Stratified K-fold cross-validation with our best set of parameters with a larger number of epochs.

#### 9.8.1 Notes on our Method
Some caveats about our methodology:
* We are using "flat" cross-validation (as was done in previous studies on this dataset). This method computationally less expensive than nested cross-validation. Flat cross-validation has the potential to overestimate of model performance. In many cases the magnitude of overestimation is small. We also mitigate this effect by using a different set of (randomly generated) fold assignments than was used for hyperparameter tuning. 
* By selecting our hyperparameters based on the smaller number of epochs (100), we favor models that are faster to to train. It is possible that using a larger number of epochs in the tuning runs would have yielded a different (and better) set of "best" hyperparameters, but would also be computationally more expensive.


#### 9.8.2 Instantiate a CrossValidatorDriver
We use a CrossValidatorDriver object to run cross-validation with a single set of hyperparameters: 

In [19]:
import lstm_adversarial_attack.tune_train.cross_validator_driver as cvd
import lstm_adversarial_attack.x19_mort_general_dataset as xmd

cv_driver = cvd.CrossValidatorDriver.from_study_path(
        device=cur_device,
        dataset=dataset,
        study_path=cfg_paths.ONGOING_TUNING_STUDY_PICKLE
    )


Lets look at the data members of `cv_driver`

In [20]:
pprint.pprint(cv_driver.__dict__)

{'dataset': <lstm_adversarial_attack.x19_mort_general_dataset.X19MGeneralDataset object at 0x7f4cfab0b280>,
 'device': device(type='cuda', index=0),
 'epochs_per_fold': 1000,
 'eval_interval': 10,
 'evals_per_checkpoint': 1,
 'hyperparameters': X19LSTMHyperParameterSettings(log_lstm_hidden_size=7,
                                                  lstm_act_name='Tanh',
                                                  dropout=0.029018875280141854,
                                                  log_fc_hidden_size=4,
                                                  fc_act_name='Tanh',
                                                  optimizer_name='Adam',
                                                  learning_rate=0.0002784280532512521,
                                                  log_batch_size=5),
 'num_folds': 5}


We will run 5-fold cross-validation using 1000 epochs per fold. We will evaluate performance and save a checkpoint once every 10 epochs. These settings are determined by the values of `CV_DRIVER_EPOCHS_PER_FOLD`, `CV_DRIVER_NUM_FOLDS`, `CV_DRIVER_EVAL_INTERVAL`, and `CV_DRIVER_EVALS_PER_CHECKPOINT` in `lstm_adversarial_attacker.config_settings`. The `.from_study_path()` class method we used to construct `cv_driver` extracts the best set of hyperparameters from `study_path` and passes them to the CrossValidationDriver constructor.

#### 9.8.3 Run Cross-Validation
We now call `cv_driver`'s `.run()` method to start the cross-validation runs.

In [ ]:
cv_driver.run()

#### 9.8.4 Monitor Cross-Validation Progress in Tensorbard
Near the start of the terminal output from the previous code cell, look for the lines:
```
Checkpoints will be saved in:
/home/devspace/project/data/cv_assessments/<timestamped_directory_name>/tensorboard
```
Then, start a zsh shell inside the app container, and launch tensorboard server:
```
$ docker exec -it lstm_aa_app /bin/zsh
$ tensorboard --logdir=/home/devspace/project/data/cv_assessments/<timestamped_directory_name>/tensorboard --host=0.0.0.0
```
The Tensorboard output can now be viewed in your browswer at http://localhost:6006

This Tensorboard screenshot was taken at the end of a 5-fold, 1000 epoch per fold cross-validation run.
![tensorboard_image](images/tensorboard_5fold_cv_best_params_1000epochs.png)

#### 9.8.5 Why Do We See Continued (Slow) Increase in Predictive Performance Up To Such High (1000) Epoch Counts?

The above AUC and validation loss curves show continued (though diminishing) improvement in predictive performance during the entire 1000 epochs. The fact that we do not observe any sign of overfitting at such a large number of epochs is somewhat unusual. A likely cause of this behavior is the `WeightedRandomSampler` used in our training `DataLoaders`. Samples with our minority class label (`mortality = 1`) only represent ~15% of the total dataset. To deal with this imbalanced dataset, we oversample from the minority class and undersample from the majority class when creating batches of samples for training. In our current implementation, some samples from the majority class go unseen by the `StandardModelTrainer` for a large number of epochs. The number of unseen samples slowly dwindles (and the amount of information available for training slowly increases), even at very high epoch counts.

#### 9.8.6 Summarize Results
We can use a CrossValidationSummarizer to identify and summarize each fold's best-performing checkpoint.

In [21]:
import lstm_adversarial_attack.tune_train.cross_validation_summarizer as cvs
cv_summarizer = cvs.CrossValidationSummarizer.from_cv_checkpoints_dir()
optimal_results_df = cv_summarizer.get_optimal_results_df(
        metric=cvs.EvalMetric.VALIDATION_LOSS,
        optimize_direction=cvs.OptimizeDirection.MIN,
    )

optimal_results_df


,fold,epoch,train_loss,validation_loss,auc,accuracy,f1,precision,recall
0,0,1000,0.349761,0.345473,0.965451,0.967849,0.967258,0.989938,0.945595
1,1,920,0.345712,0.340703,0.970717,0.972498,0.971910,0.988792,0.955595
2,2,960,0.349870,0.349510,0.961955,0.963678,0.962606,0.989406,0.937220
3,3,920,0.353771,0.348848,0.963220,0.964304,0.963820,0.987560,0.941194
4,4,790,0.347615,0.343309,0.967401,0.969995,0.968765,0.988480,0.949821


We get the mean and standard deviation of each performance metric using:

In [29]:
optimal_results_df.describe().loc[["mean", "std"], (optimal_results_df.columns != "epoch") & (optimal_results_df.columns != "fold")]

,train_loss,validation_loss,auc,accuracy,f1,precision,recall
mean,0.349346,0.345568,0.965749,0.967665,0.966872,0.988835,0.945885
std,0.003010,0.003711,0.003475,0.003742,0.003763,0.000908,0.007193


### 9.9 Comparison with Prior Work

The table below compares the predictive performance of the LSTM model in this work with other LSTM-based models using the same dataset. The current model shows the best predictive performance among all models in the table based on AUC and F1 scores. 


|  | Authors       | Model      | Input Features | AUC             | F1              | Precision       | Recall          |
|-|------------|------------|----------------|-----------------|-----------------|-----------------|-----------------|
|1 |Sun et al.  | LSTM-128 + FC-32 + FC-2 | [13 labs, 6 vitals] x 48 hr  | 0.9094 (0.0053) | 0.5429 (0.0194) | 0.4100 (0.0272) | 0.8071 (0.0269) |
|2 |Tang et al. | LSTM-256 + FC-2 | [13 labs, 6 vitals] x 48 hr + demographic data  | 0.949 (0.003) | 0.623 (0.012) | 
| 3|Tang et al. | CNN + LSTM-256 + FC-2 | [13 labs, 6 vitals] x 48 hr + demographic data | 0.940 (0.0071) | 0.633 (0.031) | 
|4 |Tang et al. | CNN + LSTM-256 + FC-2 | [13 labs, 6 vitals] x 48 hr | 0.933 (0.006) | 0.587 (0.025) |
|5 |Tang et al. | LSTM-256 + FC-2 | [13 labs, 6 vitals] x 48 hr | 0.907 (0.006) | 0.526 (0.013) |
|6 |This work   | LSTM-128 + FC-16 + FC-2 | [13 labs, 6 vitals] x 48 hr  | 0.9657 (0.0035) | 0.9669 (0.0038) | 0.9888 (0.0009) | 0.9459 (0.0072) |

> **Notes** LSTM-X indicates an LSTM with X hidden layers. FC-X indicates a fully connected layer with an output size of X. All LSTMs are bidirectional. The demographic data used in studies #2 and #3 was obtained from MIMIC-III.
 

## 10. Run Adversarial Attack Algorithm on the Trained Model

### 10.1 Adversarial Loss and Regularization
Our method of adversarial attack is similar to Chen et al.'s approach that uses an adversarial loss function and L1 regularization. When attacking a binary classification model with trained parameters $\theta$, we start with the input feature matrix $X$ of a sample that the model correctly predicts to be in class $t_{c}$, so  $M(X) = t_{c}$ where $M$ is the model's prediction function. We then search for a perturbation matrix $P$ that meets the condition:
$$
M(X + P) \ne t_{c}
$$
Since we are dealing with binary classification, this condition is equivalent to:
$$
M(X + P) = \neg{t_{c}}
$$
where $\neg{t_c}$ is the negation of $t_c$. Defining a perturbed feature matrix $\widetilde{X} = X + P$ , an adversarial loss function can be written as:
$$
max\{[Logit(\widetilde{X})]_{t_c} - [Logit(\widetilde{X})]_{\neg{t_c}}, - \kappa \}
$$

When running perturbed input $\widetilde{X}$ through a forward pass, $[Logit(\widetilde{X})]_{t_c}$ and $[Logit(\widetilde{X})]_{\neg{t_c}}$ are the **pre-activation** values at the nodes corresponding to $t_c$ and $\neg{t_c}$ the in 2-node final layer. A value $\ge 0$ is chosen for $\kappa$. Using a small non-zero value of $\kappa$ will prevent an attack algorithm from optimizing toward an infinitesimally small gap between $[Logit(\widetilde{X})]_{t_c}$ and $[Logit(\widetilde{X})]_{\neg{t_c}}$ while still targeting the small difference we want for an adversarial example.

To encourage an attack algorithm to find sparse perturbations, the following regularized version of Equation  () is used  

$$
max\{[Logit(\widetilde{X})]_{y_\theta} - [Logit(X)]_{\widetilde{y}_\theta}, - \kappa \} + \lambda||\widetilde{X}-X||_1
$$

where $\lambda$ is the L1 regularization constant. Equation () can be minimized by subgradient descent or by an Iterative Soft-Thresholding Algorithm (ISTA). The latter approach typically converges faster. 


### 10.2 Attack Algorithm and Regularization

Adversarial attacks on a particular model and dataset input features are managed by an `AdversarialAttackTrainer`. In the procedure outlined below, we discover an adversarial example any time we find $[Logit(\widetilde{X})]_{\neg{t_c}} > [Logit(\widetilde{X})]_{t_c}$, even if we have not converged near a minimum value of Equation (). We attack each batch of samples for a fixed number of iterations, regardless of how many (if any) adversarial examples are found.

1. A `LogitNoDropoutModelBuiler` creates a modified version of the target model. The modified model has all dropout probabilities set to zero, and does not have an activation function on the output layer.
2. Batches of input features are run through a `FeaturePerturber` (implemented in `attack.feature_perturber`) that generates slightly modified versions of original features
3. The perturbed features are run through the modified model that was built by the `LogitNoDropoutModelBuiler` to obtain values for $[Logit(\widetilde{X})]_{t_c}$ and $[Logit(\widetilde{X})]_{\neg{t_c}}$
4. An instance of custom PyTorch loss function ` AdversarialLoss`, which implements Equation (), calculates a loss tensor
5. The Pytorch `.backward()`  method of the loss tensor finds the gradient of the loss with respect to the elements of the `FeaturePerturber.perturbation` tensor

6. If the current $Logit$ values resulting from a sample's perturbed input features represent an adversarial example, and the example is either the first or lowest loss example for that sample, the perturbations and other details are stored in a `BatchResult` object.

7. A Pytorch optimizer uses the loss gradient to calculate and apply adjustments to the perturbations

8. The `AdversarialAttackTrainer.apply_soft_bounded_threshold()` method performs ISTA thresholding on the perturbations

9. The perturbations (which have been adjusted by the optimizer *and* ISTA thresholding, are used in step 1 of the next attack iteration.

Two key points from above procedure are: (1) Unlike the method used in [], we do not stop attacking an example upon finding a single adversarial perturbation for it.  (2) We use a combination of subgradient descent (in step 7), and ISTA (in step 8) to minimize (or at least reduce) the value of equation (). We do not know if this approach is guaranteed to converge to a minimum in the adversarial loss function, but empirically, we find this subgradient descent + ISTA more effective at finding sparse adversarial examples than either method is on its own.




## 10.3 Attack Hyperparameter Tuning

In [ ]:
from lstm_adversarial_attack.attack import tune_attacks as tua

initial_attack_tuning_study = tua.start_new_tuning(
        num_trials=50,
        target_model_assessment_type=amr.ModelAssessmentType.KFOLD,
        objective=aht.AttackTunerObjectivesBuilder.sparse_small_max(),
    )


In [ ]:
# continued_study = resume_tuning(num_trials=50)